# Statistical Model Languages

Tarea 3 - Luis Eduardo Robles Jimenez

Times

|Date                           |Time                           |Task                           |                   
|---                            |---                            |---                            | 
|Mar 25                         | 11:50 - 12:20                 | Read hw and outlined notebook |
|Mar 27                         | 12:55 - 01:42                 | Preprocessing                 |
|Mar 27                         | 02:50 - 03:10                 | More preprocessing            |
|Apr 01                         | 09:52 -               ||

## Language Model and Evaluation

In [55]:
import nltk
from nltk.tokenize import TweetTokenizer

### 1. Preprocessing

In [56]:
tokens = dict()
tokens['begin'], tokens['end'], tokens['unknown'] = '<s>', '</s>', '<unk>'

In [57]:
def _create_vocabulary(corpus, size): # Returns words sorted by frequency
    words, tokenizer, corpusByWords = [], TweetTokenizer(), []
    for doc in corpus:
        tokens = tokenizer.tokenize(doc)
        words += tokens
        corpusByWords += [tokens]
    count = nltk.FreqDist(words)
    count = sorted([(count[key], key) for key in count])[::-1][:size]
    return [word for _, word in count], corpusByWords

def load_corpus(corpus_select = "tweets", vocabSize = 100):
    corpus = []
    path_corpus = "../../data/agresividad/mex_train.txt"

    with open(path_corpus, "r") as f_corpus:
        for tuit in f_corpus:
            corpus += [tuit[:-1]]
    
    vocab, tokenized = _create_vocabulary(corpus, vocabSize)
    corpus = []
    for doc in tokenized:
        tweet = []
        tweet.append(tokens['begin'])
        for word in doc: 
            tweet.append(tokens['unknown'] if word.lower() not in vocab else word)
        tweet.append(tokens['end'])
        corpus.append(tweet)
    return corpus

In [58]:
corpus = load_corpus()

#### Comment

### 2. Models Training

In [59]:
class LanguageModel:
    def __init__(self, corpus = None):
        self.corpus = corpus
        self.nGrams = dict()
    def P(self, *words):
        pass
    def _GoodTuring():
        pass
class Unigram(LanguageModel):
    def __init__(self, corpus = None):
        super().__init__(corpus)
        c = 0
        for line in corpus:
            for word in line:
                if not word in self.nGrams: self.nGrams[word] = 0
                self.nGrams[word] += 1
                c += 1
        for word in self.nGrams:
            self.nGrams[word] /= c
    def P(self, *words):    # Only one n-gram
        assert len(words) == 1, 'Trying to calculate the probability of more than one word'
        word = words[0] if words[0] in self.nGrams else tokens['unknown']
        return self.nGrams[word]

In [60]:
uni = Unigram(corpus)

In [61]:
print(uni.P("es"))
print(uni.P("otorrino"))

0.007708722325953587
0.19280697074775496


#### Comment

### 3. Interpolated Model

#### Comment

## Text Generation

### 1. Tweet Functionality

#### Comment

### 2. AMLO model

#### Comment

### 3. Evaluation with custom phrases

#### Comment

### 4. More evaluation 

#### Comment

## El ahorcado

### 1. Norvig's Hangman

#### Comment

### 2. Follow-up

#### Comment